<a href="https://colab.research.google.com/github/esalimi/Kaggle_NLI/blob/master/Watson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Step 1: Building the BERT Model**



In [ ]:
!pip install keras-bert

  Created wheel for keras-bert: filename=keras_bert-0.86.0-cp36-none-any.whl size=34145 sha256=42badc02b48a28812e3a1c424b9d3c20d9eb3df1acff4372af1eba4931533da3
  Stored in directory: /root/.cache/pip/wheels/66/f0/b1/748128b58562fc9e31b907bb5e2ab6a35eb37695e83911236b
  Created wheel for keras-transformer: filename=keras_transformer-0.38.0-cp36-none-any.whl size=12942 sha256=8e36dc3de66ddaa211cff8ff74b6e839f5b32d2ecdf07d37be8a58ee2ff553bb
  Stored in directory: /root/.cache/pip/wheels/e5/fb/3a/37b2b9326c799aa010ae46a04ddb04f320d8c77c0b7e837f4e
  Created wheel for keras-pos-embd: filename=keras_pos_embd-0.11.0-cp36-none-any.whl size=7554 sha256=9e63d64a0750766c5617bc200bbfd06f50ade55d0e79323ac81fc74589f5afec
  Stored in directory: /root/.cache/pip/wheels/5b/a1/a0/ce6b1d49ba1a9a76f592e70cf297b05c96bc9f418146761032
  Created wheel for keras-multi-head: filename=keras_multi_head-0.27.0-cp36-none-any.whl size=15612 sha256=03563de335e098dc843ebe2330a7063d689321bd8b088fbb3b3990cb79ad41df
  Stor

In [ ]:
# Importing required packages keras, keras_bert, os
import os
import json
import keras
from keras import layers

# Importing required packages from keras_bert
from keras_bert.loader import load_trained_model_from_checkpoint
from keras_bert.bert import *
from keras_bert import extract_embeddings 

In [ ]:
# Loading bert_base from google drive
from google.colab import drive
drive.mount('/content/drive')
model_path = '/content/drive/My Drive/uncased_L-12_H-768_A-12'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Testing the import
texts = ['all work and play', 'makes jack a dull boy~']
embeddings = extract_embeddings(model_path, texts)
embedding_sentence1 = embeddings[0]
embedding_sentence2 = embeddings[1]

print(len(embedding_sentence1[0]))
print(embeddings[0])

768
[[-0.3183194   0.17749757  0.14557648 ... -0.47990972 -0.685876
  -0.14804517]
 [-0.2539365   0.20985767  0.24948463 ... -0.24461871 -0.4896369
  -0.24464582]
 [ 0.08667843  0.17276856  0.31595454 ... -0.75597656 -0.7137721
  -0.20142424]
 [ 0.1849249   0.46440715 -0.4347621  ...  0.4690967   0.11448962
  -0.4629252 ]
 [ 0.25327638 -0.06883571 -0.16352269 ... -1.0589056  -0.70101875
   0.05821889]
 [-0.06629732  1.3095123   0.25281972 ... -1.1509613  -0.30832586
  -0.14396843]]


In [ ]:
# Function to define the structure of the bert

def get_bert_model(model_path, trainable = True, max_tokens=512, mode = 'pair'):
  config_file = os.path.join(model_path, 'bert_config.json')
  checkpoint_file = os.path.join(model_path, 'bert_model.ckpt')
  bert = load_trained_model_from_checkpoint(config_file, checkpoint_file, training=False, trainable=True)
  """
  The pair mode gets 2 sntcs as input: premise & hypothesis.
  Using Lambda layers: Lambda layer is an easy way to customize a layer to do simple arithmetic
  """
  if mode == 'pair':
    input_tensor = layers.Input(shape=(2,max_tokens, ))
    word_tokens = layers.Lambda(lambda x: x[:,0, :])(input_tensor)
    segment_encode = layers.Lambda(lambda x: x[:,1,:])(input_tensor)
  if mode == 'single':
    input_tensor = layers.Input(shape=(max_tokens, ))
    word_tokens = input_tensor
    segment_encode = layers.Lambda(lambda x: x*0)(input_tensor) #seems
  
  output0 = bert([word_tokens, segment_encode])
  output1 = layers.Lambda(lambda x: x[:,0,:])(output0) #0-> cls : [cls]+...[SEP],...,[sep]
  output = layers.Dense(768)(output1)
  final_output = layers.Dense(3, activation='softmax')(output)

  model = keras.models.Model(inputs =input_tensor , outputs =final_output )
  print(model.summary)
  return model

In [ ]:
# Test for model installation: calling the function to print model summary
model = get_bert_model(model_path)
print(model.summary())

<bound method Model.summary of <tensorflow.python.keras.engine.functional.Functional object at 0x7f5bc80b0748>>
Model: "functional_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 2, 512)]     0                                            
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 512)          0           input_1[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 512)          0           input_1[0][0]                    
__________________________________________________________________________________________________
functional_7 (Functional)       (None, 512, 768)     108891648   lambda[0]

# **Step 2: Importing Data from Google Drive**

In [1]:
# Importing required packages
import pandas as pd
from sklearn import preprocessing

In [5]:
# reading the training data from google drive: English senteces so far
data =  pd.read_csv('/content/drive/My Drive/train.csv')
data.head()

# Picking only english sentences
data_en = data[data['language']=='English']
data_en.head()


,id,premise,hypothesis,lang_abv,language,label
0,5130fd2cb5,and these comments were considered in formulat...,The rules developed in the interim were put to...,en,English,0
1,5b72532a0b,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,en,English,2
3,5622f0c60b,you know they can't really defend themselves l...,They can't defend themselves because of their ...,en,English,0
7,fdcd1bd867,From Cockpit Country to St. Ann's Bay,From St. Ann's Bay to Cockpit Country.,en,English,2
8,7cfb3d272c,"Look, it's your skin, but you're going to be i...",The boss will fire you if he sees you slacking...,en,English,1


In [3]:
# Training data set: Input and labels
X_train = data_en[['premise','hypothesis']]
labels = data_en['label']

#Need to convert labels (0,1,2) into binary labels
LB = preprocessing.LabelBinarizer()
y_train = LB.fit_transform(labels)


In [4]:
# Printing data to verify the import process
X_train.head()
y_train[:10]

array([[1, 0, 0],
       [0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 1, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 0, 1],
       [0, 0, 1]])

# **Step 3: Fine Tuning Bert**


In [ ]:
import numpy as np
import codecs
from keras_bert import load_trained_model_from_checkpoint

token_dict = {}
with codecs.open('/content/drive/My Drive/uncased_L-12_H-768_A-12/vocab.txt', 'r', 'utf8') as reader:
    for line in reader:
        token = line.strip()
        token_dict[token] = len(token_dict)

In [ ]:
from keras_bert import Tokenizer
tokenizer = Tokenizer(token_dict)
  

In [ ]:
def bertEncoder(sentences_df,max_len =512):
  """
  inputs: a dataframe of form df[sentence1,sentence2]
  output : bert encoded in the form of [indeces,segments]
  Note: for pretraining you will need Mask input as well.
  """
  encoded_data = []
  for idx, row in sentences_df.iterrows():
    indices, segments = tokenizer.encode(first=row[0], second=row[1], max_len=512)
    encoded_data.append([indices,segments])
    # encode_seg.append(segments)
  return np.array(encoded_data)

# **Step 4: Train Data**

In [ ]:
train_data= bertEncoder(X_train, max_len=512)

In [ ]:
len(train_data[0])
np.shape(train_data),np.shape(y_train)

((6870, 2, 512), (6870, 3))